In [2]:
import sys
import argparse
import os
import pandas as pd
import numpy as np
import re
import math
from collections import Counter

sys.path.append('/Users/friedman/Desktop/hypermutationProjectFinal/scripts/utilityScripts')
import configuration_util

filePathDict = configuration_util.get_all_files_path_dict()
import analysis_utils
import get_gene_and_cohort_list_utils

In [46]:
def summarize_inactivation_method(maf, hyperIds, normalIds, cancerTypes = ['']):
    listOfDicts = []
    for cancerType in cancerTypes:
        cancerTypeMaf = maf[maf['cancerType'] == cancerType]
        hyperMaf = cancerTypeMaf[cancerTypeMaf['Tumor_Sample_Barcode'].isin(hyperIds)]
        normalMaf = cancerTypeMaf[cancerTypeMaf['Tumor_Sample_Barcode'].isin(normalIds)]
    
        listOfDicts.append({'nHyperLOH': len(set(hyperMaf[hyperMaf['lossType'] == 'LOH']['geneCase'])),
                            'nHyperComposite': len(set(hyperMaf[hyperMaf['lossType'] == 'composite_mutation']['geneCase'])),
                            'nNormalLOH': len(set(normalMaf[normalMaf['lossType'] == 'LOH']['geneCase'])),
                            'nNormalComposite': len(set(normalMaf[normalMaf['lossType'] == 'composite_mutation']['geneCase'])),
                            'cancerType': cancerType
                           })
        
    df = pd.DataFrame(listOfDicts)
    df['totalHyper'] = df['nHyperLOH'] + df['nHyperComposite']
    df['totalNormal'] = df['nNormalLOH'] + df['nNormalComposite']
        
    return df

    

In [47]:
allImpactMutsMaf = pd.read_table(filePathDict['IMPACT_BASE_MAF'])

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [48]:
allImpactMutsMaf['varUuid'] = allImpactMutsMaf.apply(lambda row:
    row['Tumor_Sample_Barcode'] + '_' + str(row['Start_Position']) + '_' + str(row['Tumor_Seq_Allele2']), axis=1)
oncogenicMutIds = set(allImpactMutsMaf[allImpactMutsMaf['oncogenic'].notnull()]['varUuid'])

In [49]:
mafWithClonalityAnnotation = pd.read_csv(filePathDict['IMPACT_MAF_WITH_ADJUSTED_CLONALITY_ANNOTATION'])
mafWithClonalityAnnotation['varUuid'] = mafWithClonalityAnnotation.apply(lambda row:
    row['Tumor_Sample_Barcode'] + '_' + str(row['Start_Position']) + '_' + str(row['Tumor_Seq_Allele2']), axis=1)

In [50]:
hypermutantIds = get_gene_and_cohort_list_utils.get_all_hypermutant_ids()
normalIds = get_gene_and_cohort_list_utils.get_all_normal_ids()

In [51]:
#mafWithClonalityAnnotation[['tcn', 'lcn']]
tsgs = get_gene_and_cohort_list_utils.get_tsgs()
tsgOncogenicMaf = mafWithClonalityAnnotation[(mafWithClonalityAnnotation['varUuid'].isin(oncogenicMutIds)) &
                                             (mafWithClonalityAnnotation['Hugo_Symbol'].isin(tsgs))]

In [52]:
tsgOncogenicMaf['geneCase'] = tsgOncogenicMaf.apply(lambda row: row['Hugo_Symbol'] + '_' + row['Tumor_Sample_Barcode'], axis=1)
occurenceDict = dict(tsgOncogenicMaf['geneCase'].value_counts())                                                                                     
tsgOncogenicMaf['isMultiplet'] = tsgOncogenicMaf['geneCase'].apply(lambda x: True if occurenceDict[x] > 1 else False)
tsgOncogenicMaf['isLOH'] = tsgOncogenicMaf['lcn'].apply(lambda x: True if x == 0 else False)
tsgOncogenicMaf['lossType'] = tsgOncogenicMaf.apply(lambda row: 'LOH' if row['isLOH'] == True
    else 'composite_mutation' if row['isMultiplet'] == True else False, axis=1)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

In [53]:
cancerTypeDict = get_gene_and_cohort_list_utils.get_impact_cancer_type_info()
tsgOncogenicMaf['cancerType'] = tsgOncogenicMaf['Tumor_Sample_Barcode'].apply(lambda x:
    cancerTypeDict[x] if x in cancerTypeDict else None)


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [54]:
df = summarize_inactivation_method(tsgOncogenicMaf, hypermutantIds, normalIds, cancerTypes =
    ['Endometrial Cancer', 'Colorectal Cancer', 'Melanoma',
     'Prostate Cancer', 'Glioma', 'Bladder Cancer', 'Non-Small Cell Lung Cancer'])

In [55]:
df.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/inactivationMethod.tsv', index=False, sep='\t')